# 数据预处理

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
df = pd.read_csv('data.csv')
df = df.dropna(axis=0)
df

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,dt,kdcode,volume,turnover,turn,pct_chg,vwap,trade_status,adj_close,adj_open,adj_high,adj_low,label
0,2016-01-04,000001.SZ,56349787.0,6.603762e+08,0.4774,-5.51,11.719231,0,935.197763,990.500720,992.976972,926.943591,0.0
1,2016-01-05,000001.SZ,66326995.0,7.555314e+08,0.5619,0.61,11.391008,0,940.975684,930.245260,955.007778,920.340253,1.0
2,2016-01-06,000001.SZ,51570644.0,5.916985e+08,0.4369,1.14,11.473553,0,951.706109,942.626519,954.182361,940.150267,0.0
3,2016-01-07,000001.SZ,17476110.0,1.948695e+08,0.1481,-5.12,11.150622,0,903.006490,941.801102,941.801102,900.530238,0.0
4,2016-01-08,000001.SZ,74752758.0,8.313345e+08,0.6333,1.64,11.121122,0,917.864001,925.292756,931.896094,899.704821,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680759,2021-07-26,688981.SH,97665400.0,5.346218e+09,5.5900,10.25,54.740148,0,57.200000,51.850000,57.320000,51.120000,1.0
4680760,2021-07-27,688981.SH,308012900.0,1.993186e+10,16.4600,12.59,64.711114,0,64.400000,57.400000,68.640000,57.400000,1.0
4680761,2021-07-28,688981.SH,172850500.0,1.044512e+10,9.2400,-7.98,60.428625,0,59.260000,62.950000,63.490000,58.280000,0.0
4680762,2021-07-29,688981.SH,144274400.0,8.933641e+09,7.7100,5.08,61.921177,0,62.270000,60.400000,64.580000,59.370000,1.0


## 1. 根据股票代码划分数据

In [2]:
stocks_code = df['kdcode'].unique()
stocks_num = len(stocks_code)
"股票数量",stocks_num

('股票数量', 4022)

In [3]:
# 根据股票代码划分数据
for i, stock_i in enumerate(tqdm(stocks_code)):
    stock_i_data = df[df['kdcode'].isin([stock_i])]
    exec("df%s = stock_i_data" % i)

100%|██████████| 4022/4022 [03:34<00:00, 18.72it/s]


In [4]:
res_df = pd.DataFrame()

## 2. 按照股票添加特征

### 2.1 添加1日、5日、10日、20日`vwap`均价特征

In [5]:
sizes = [5, 10, 20]
res_df['vwap'] = df['vwap']

In [6]:
for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        vwap = stock['vwap'].values # 成交价
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = vwap[:index + 1].mean()
            else:
                mean_res = vwap[index - size + 1:index + 1].mean()
            exec("meanList%s.append(%s)" % (t, mean_res))

res_df['vwap_mean5'] = meanList0
res_df['vwap_mean10'] = meanList1
res_df['vwap_mean20'] = meanList2

遍历所有股票: 100%|██████████| 4022/4022 [01:20<00:00, 49.69it/s] 


### 2.2 添加3日、5日、10日平均涨跌幅`pct_chg`

In [7]:
sizes = [3, 5, 10]
for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        pct_chg = stock['pct_chg'].values # 涨跌幅
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = pct_chg[:index + 1].mean()
            else:
                mean_res = pct_chg[index - size + 1:index + 1].mean()
            exec("meanList%s.append(%s)" % (t, mean_res))

res_df['chg_mean3'] = meanList0
res_df['chg_mean5'] = meanList1
res_df['chg_mean10'] = meanList2

遍历所有股票: 100%|██████████| 4022/4022 [01:25<00:00, 46.79it/s] 


In [8]:
res_df

,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10
0,11.719231,11.719231,11.719231,11.719231,-5.510000,-5.510000,-5.510000
1,11.391008,11.555119,11.555119,11.555119,-2.450000,-2.450000,-2.450000
2,11.473553,11.527931,11.527931,11.527931,-1.253333,-1.253333,-1.253333
3,11.150622,11.433604,11.433604,11.433604,-1.123333,-2.220000,-2.220000
4,11.121122,11.371107,11.371107,11.371107,-0.780000,-1.448000,-1.448000
...,...,...,...,...,...,...,...
4680759,54.740148,52.787750,53.961543,56.229254,2.966667,2.088000,0.170000
4680760,64.711114,55.416810,54.720099,56.497093,6.910000,4.576000,1.353000
4680761,60.428625,57.019739,55.101627,56.441425,4.953333,2.702000,0.737000
4680762,61.921177,58.804745,55.757486,56.499986,3.230000,3.566000,1.393000


### 2.3 添加3日、5日、10日平均开盘价

In [9]:
sizes = [3, 5, 10]
for t, size in enumerate(sizes):
    exec("meanList%s = []" % t)
    for i in tqdm(range(stocks_num), desc='遍历所有股票'): # 循环股票
        exec("stock = df%s" % i)
        adj_open = stock['adj_open'].values # 开盘价
        for index in range(stock.shape[0]):
            if index < size - 1:
                mean_res = adj_open[:index + 1].mean()
            else:
                mean_res = adj_open[index - size + 1:index + 1].mean()
            exec("meanList%s.append(%s)" % (t, mean_res))

res_df['open_mean3'] = meanList0
res_df['open_mean5'] = meanList1
res_df['open_mean10'] = meanList2

遍历所有股票: 100%|██████████| 4022/4022 [01:24<00:00, 47.64it/s] 


In [10]:
res_df

,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10
0,11.719231,11.719231,11.719231,11.719231,-5.510000,-5.510000,-5.510000,990.500720,990.500720,990.500720
1,11.391008,11.555119,11.555119,11.555119,-2.450000,-2.450000,-2.450000,960.372990,960.372990,960.372990
2,11.473553,11.527931,11.527931,11.527931,-1.253333,-1.253333,-1.253333,954.457500,954.457500,954.457500
3,11.150622,11.433604,11.433604,11.433604,-1.123333,-2.220000,-2.220000,938.224293,951.293400,951.293400
4,11.121122,11.371107,11.371107,11.371107,-0.780000,-1.448000,-1.448000,936.573459,946.093271,946.093271
...,...,...,...,...,...,...,...,...,...,...
4680759,54.740148,52.787750,53.961543,56.229254,2.966667,2.088000,0.170000,52.496667,52.078000,53.945000
4680760,64.711114,55.416810,54.720099,56.497093,6.910000,4.576000,1.353000,54.083333,53.358000,54.024000
4680761,60.428625,57.019739,55.101627,56.441425,4.953333,2.702000,0.737000,57.400000,55.568000,54.614000
4680762,61.921177,58.804745,55.757486,56.499986,3.230000,3.566000,1.393000,60.250000,57.120000,55.052000


## 3. 特征处理

* **通过取对数log缩小数据差距**

open_mean的最小值与最大值差距过大

In [27]:
res_df['open_mean3'].min()

1.0868

In [28]:
res_df['open_mean3'].max()

237952.08

In [29]:
res_df['open_mean3'] = np.log(res_df['open_mean3'])
res_df['open_mean5'] = np.log(res_df['open_mean5'])
res_df['open_mean10'] = np.log(res_df['open_mean10'])
res_df

,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10
0,11.719231,11.719231,11.719231,11.719231,-5.510000,-5.510000,-5.510000,6.898211,6.898211,6.898211
1,11.391008,11.555119,11.555119,11.555119,-2.450000,-2.450000,-2.450000,6.867322,6.867322,6.867322
2,11.473553,11.527931,11.527931,11.527931,-1.253333,-1.253333,-1.253333,6.861143,6.861143,6.861143
3,11.150622,11.433604,11.433604,11.433604,-1.123333,-2.220000,-2.220000,6.843989,6.857823,6.857823
4,11.121122,11.371107,11.371107,11.371107,-0.780000,-1.448000,-1.448000,6.842228,6.852341,6.852341
...,...,...,...,...,...,...,...,...,...,...
4680759,54.740148,52.787750,53.961543,56.229254,2.966667,2.088000,0.170000,3.960750,3.952743,3.987965
4680760,64.711114,55.416810,54.720099,56.497093,6.910000,4.576000,1.353000,3.990526,3.977024,3.989428
4680761,60.428625,57.019739,55.101627,56.441425,4.953333,2.702000,0.737000,4.050044,4.017607,4.000290
4680762,61.921177,58.804745,55.757486,56.499986,3.230000,3.566000,1.393000,4.098503,4.045154,4.008278


In [32]:
res_df['vwap'].max()

2554.307336

* **通过删除$3\sigma$外的数据**

## 4. 归一化

In [33]:
tmp_df = res_df.copy()

In [34]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
_df = min_max_scaler.fit_transform(res_df)
_df = pd.DataFrame(_df, columns=res_df.columns)
_df

,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10
0,0.004588,0.004771,0.004929,0.005196,0.016312,0.016312,0.016312,0.554217,0.553924,0.552504
1,0.004460,0.004704,0.004860,0.005123,0.019133,0.019133,0.019133,0.551705,0.551410,0.549983
2,0.004492,0.004693,0.004849,0.005111,0.020236,0.020236,0.020236,0.551202,0.550908,0.549478
3,0.004365,0.004655,0.004809,0.005069,0.020356,0.019345,0.019345,0.549807,0.550637,0.549207
4,0.004354,0.004629,0.004783,0.005042,0.020673,0.020057,0.020057,0.549664,0.550191,0.548760
...,...,...,...,...,...,...,...,...,...,...
4680759,0.021431,0.021490,0.022697,0.024931,0.024127,0.023317,0.021548,0.315332,0.314231,0.314924
4680760,0.025334,0.022560,0.023016,0.025050,0.027763,0.025611,0.022639,0.317754,0.316207,0.315044
4680761,0.023658,0.023212,0.023177,0.025025,0.025959,0.023883,0.022071,0.322594,0.319510,0.315930
4680762,0.024242,0.023939,0.023453,0.025051,0.024370,0.024680,0.022676,0.326535,0.321752,0.316582


In [35]:
# 添加不需归一化的列
_df['label'] = df['label']
_df['kdcode'] = df['kdcode']
_df['dt'] = df['dt']
order = ['dt','kdcode','vwap','vwap_mean5','vwap_mean10','vwap_mean20','chg_mean3','chg_mean5','chg_mean10','open_mean3','open_mean5','open_mean10','label']
df_ = _df[order]
df_

,dt,kdcode,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10,label
0,2016-01-04,000001.SZ,0.004588,0.004771,0.004929,0.005196,0.016312,0.016312,0.016312,0.554217,0.553924,0.552504,0.0
1,2016-01-05,000001.SZ,0.004460,0.004704,0.004860,0.005123,0.019133,0.019133,0.019133,0.551705,0.551410,0.549983,1.0
2,2016-01-06,000001.SZ,0.004492,0.004693,0.004849,0.005111,0.020236,0.020236,0.020236,0.551202,0.550908,0.549478,0.0
3,2016-01-07,000001.SZ,0.004365,0.004655,0.004809,0.005069,0.020356,0.019345,0.019345,0.549807,0.550637,0.549207,0.0
4,2016-01-08,000001.SZ,0.004354,0.004629,0.004783,0.005042,0.020673,0.020057,0.020057,0.549664,0.550191,0.548760,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680759,2021-07-26,688981.SH,0.021431,0.021490,0.022697,0.024931,0.024127,0.023317,0.021548,0.315332,0.314231,0.314924,1.0
4680760,2021-07-27,688981.SH,0.025334,0.022560,0.023016,0.025050,0.027763,0.025611,0.022639,0.317754,0.316207,0.315044,1.0
4680761,2021-07-28,688981.SH,0.023658,0.023212,0.023177,0.025025,0.025959,0.023883,0.022071,0.322594,0.319510,0.315930,0.0
4680762,2021-07-29,688981.SH,0.024242,0.023939,0.023453,0.025051,0.024370,0.024680,0.022676,0.326535,0.321752,0.316582,1.0


## 5. 分开训练集和测试集

In [36]:
df_.set_index(['dt'], inplace = True)
df_.index = pd.to_datetime(df_.index)
df_

,kdcode,vwap,vwap_mean5,vwap_mean10,vwap_mean20,chg_mean3,chg_mean5,chg_mean10,open_mean3,open_mean5,open_mean10,label
dt,,,,,,,,,,,,
2016-01-04,000001.SZ,0.004588,0.004771,0.004929,0.005196,0.016312,0.016312,0.016312,0.554217,0.553924,0.552504,0.0
2016-01-05,000001.SZ,0.004460,0.004704,0.004860,0.005123,0.019133,0.019133,0.019133,0.551705,0.551410,0.549983,1.0
2016-01-06,000001.SZ,0.004492,0.004693,0.004849,0.005111,0.020236,0.020236,0.020236,0.551202,0.550908,0.549478,0.0
2016-01-07,000001.SZ,0.004365,0.004655,0.004809,0.005069,0.020356,0.019345,0.019345,0.549807,0.550637,0.549207,0.0
2016-01-08,000001.SZ,0.004354,0.004629,0.004783,0.005042,0.020673,0.020057,0.020057,0.549664,0.550191,0.548760,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-26,688981.SH,0.021431,0.021490,0.022697,0.024931,0.024127,0.023317,0.021548,0.315332,0.314231,0.314924,1.0
2021-07-27,688981.SH,0.025334,0.022560,0.023016,0.025050,0.027763,0.025611,0.022639,0.317754,0.316207,0.315044,1.0
2021-07-28,688981.SH,0.023658,0.023212,0.023177,0.025025,0.025959,0.023883,0.022071,0.322594,0.319510,0.315930,0.0


In [37]:
dateSepVal = pd.to_datetime("2020-01-01")
train_df = df_[df_.index < dateSepVal]
test_df = df_[df_.index > dateSepVal]

## 6. 保存到文件

In [38]:
train_df.to_csv('trainData2.csv')

In [39]:
test_df.to_csv('testData2.csv')

TODO:
* 选股？